In [1]:
"""
File: tng_find_targets.py
Author: Matthew Ogden
Email: ogdenm12@gmail.com
Github: mbogden
Created: 2023-Nov-09

Description: 
    This code is designed to interact with the IllustrisTNG Simulation Data and related catalogs. 
    It's goal is to identify close interactions/mergers between two galaxies.

References:  
- TNG50 Simulation Data
    - Nelson, D. et al. (2015). The Illustris Simulation: Public Data Release. Astronomy and Computing, 13, 12–37. https://doi.org/10.1016/j.ascom.2015.09.003
    - Nelson, D. et al. (2019). First results from the TNG50 simulation: Galactic outflows driven by supernovae and black hole feedback. Monthly Notices of the Royal Astronomical Society, 490(3), 3234–3261. https://doi.org/10.1093/mnras/stz2306
    - Pillepich, A. et al. (2019). First results from the TNG50 simulation: The evolution of stellar and gaseous discs across cosmic time. Monthly Notices of the Royal Astronomical Society, 490(3), 3196–3233. https://doi.org/10.1093/mnras/stz2338
    
- Subhalo Morphology (Deep Learning)
    - Huertas-Company, M.et al. (2019). The Hubble Sequence at $z\sim0$ in the IllustrisTNG simulation with deep learning. Monthly Notices of the Royal Astronomical Society, 489(2), 1859–1879. https://doi.org/10.1093/mnras/stz2191
    - Varma, S., Huertas-Company, M., Pillepich, A., Nelson, D., Rodriguez-Gomez, V., Dekel, A., Faber, S. M., Iglesias-Navarro, P., Koo, D. C., & Primack, J. (2021). The building up of observed stellar scaling relations of massive galaxies and the connection to black hole growth in the TNG50 simulation. Monthly Notices of the Royal Astronomical Society, 509(2), 2654–2673. https://doi.org/10.1093/mnras/stab3149
    
Acknowledgements:    
- Sections of this code were written with the assistance of ChatGPT made by OpenAI.

- The IllustrisTNG simulations were undertaken with compute time awarded by 
    the Gauss Centre for Supercomputing (GCS) under GCS Large-Scale Projects GCS-ILLU 
    and GCS-DWAR on the GCS share of the supercomputer Hazel Hen 
    at the High Performance Computing Center Stuttgart (HLRS), 
    as well as on the machines of the Max Planck Computing and Data Facility (MPCDF) in Garching, Germany.
"""


"\nFile: tng_find_targets.py\nAuthor: Matthew Ogden\nEmail: ogdenm12@gmail.com\nGithub: mbogden\nCreated: 2023-Nov-09\n\nDescription: \n    This code is designed to interact with the IllustrisTNG Simulation Data and related catalogs. \n    It's goal is to identify close interactions/mergers between two galaxies.\n\nReferences:  \n- TNG50 Simulation Data\n    - Nelson, D. et al. (2015). The Illustris Simulation: Public Data Release. Astronomy and Computing, 13, 12–37. https://doi.org/10.1016/j.ascom.2015.09.003\n    - Nelson, D. et al. (2019). First results from the TNG50 simulation: Galactic outflows driven by supernovae and black hole feedback. Monthly Notices of the Royal Astronomical Society, 490(3), 3234–3261. https://doi.org/10.1093/mnras/stz2306\n    - Pillepich, A. et al. (2019). First results from the TNG50 simulation: The evolution of stellar and gaseous discs across cosmic time. Monthly Notices of the Royal Astronomical Society, 490(3), 3196–3233. https://doi.org/10.1093/mnra

# Finding Galaxy Mergers within the IllustrisTNG Simulation

## Imports

In [2]:
# ================================ IMPORTS ================================ #
import os, argparse, h5py
import numpy as np, pandas as pd, scipy.signal
import matplotlib.pyplot as plt 
import illustris_python as il
import tng_functions as tf
from collections import Counter

print("Imports Done")

# Global variables
SIM_DIR = '/home/tnguser/sims.TNG/TNG50-1/output/'

# A useful fucntion I often use for indented printing
def tabprint( printme, start = '\t - ', end = '\n' ):
    print( start + str(printme), end = end )

Imports Done


---
## Command Line Arguments

This is written in JupyterLab, and will be compiled and ran in python for faster execution.  This will define the possible input command line arguements.


WARNING:  I have not been consistent with implementing and following arguments.  Code still in indevlopment.  

In [3]:
# This argument decides if code is in python or jupyter.
buildEnv = False

# Define argument parser function 
def initParser():
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument( '-s', '--simDir', default = '/home/tnguser/sims.TNG/TNG50-1/output/',  type=str, \
                        help="Base directory for a single simulation on the IllustrisTNG servers.")   
    
    parser.add_argument( '-n', '--simName', default = 'TNG50-1',  type=str, \
                        help="Name for the simulation being worked on.")
    
    parser.add_argument( '-o', '--overwrite', default = False,  type=bool, \
                        help="Overwrite output files?  If false, will check if output file exists before beginning time-consuming tasks.")
    
    parser.add_argument( '-t', '--trim', default = -1,  type=int, \
                        help="Default number of subhalos to consider, sorted by highest mass first.")
    
    parser.add_argument( '-f', '--function', default = 'None', type=str, \
                        help="Default function program will be executing.")
    
    parser.add_argument( '-d', '--dataDir', default = 'data', type=str, \
                        help="Default location to store misc data files.")

    return parser

parser = initParser()
print("Args: Defined")

Args: Defined


## To Python? Or to JupyterLab? 
This will establish if this is being run in a JupyterLab environment or from Command Line in Python. 

NOTE:  If you're running this in Jupyter, modify the `cmdStr` below to whatever variables you need.

In [4]:
# Am I in a jupyter notebook?
try:
    
    # This command is NOT available in a python script
    get_ipython().__class__.__name__
    buildEnv = True
    print ("In Building Environment")
    
    # Command Line Arguments
    cmdStr  = 'python3 targets-working.py'
    cmdStr += ' --trim 10'
    cmdStr += ' --dataDir tng-data'
    
    # Read string as if command line
    print( "CMD Line: \n\t$:", cmdStr)
    
    # This function doesn't like the 'python3 file.py' part.
    args = parser.parse_args(cmdStr.split()[2:])

# Or am I in a python script?
except:
    
    # Read CMD arguments
    args = parser.parse_args()
    

print( "Args: Read")
print( args )

# Setup data directory if not found
os.makedirs(args.dataDir, exist_ok=True)

In Building Environment
CMD Line: 
	$: python3 targets-working.py --trim 10 --dataDir tng-data
Args: Read
Namespace(simDir='/home/tnguser/sims.TNG/TNG50-1/output/', simName='TNG50-1', overwrite=False, trim=10, function='None', dataDir='tng-data')


In [5]:
if buildEnv: 
    # Location of one simulation
    print("Is this locational valid?")
    print( f"Simulation data: {os.path.exists( args.simDir )} - {args.simDir}" )

Is this locational valid?
Simulation data: True - /home/tnguser/sims.TNG/TNG50-1/output/


---
# Halos and SubHalos
Within the simulation, Halos are the largest set of objects that are gravitationally bound to each other, I like to think of them as galaxy clusters.  Subhalos are also gravitationally bound objects but more dense, and I suspect has to do with potential energy.   I like to think of them as galaxies, globular clusters, blobs of intergalactic gas, etc.  (That's my reasoning and I'm sticking to it)


For more information, pleas visit the IllustrisTNG Data Specification Page.  https://www.tng-project.org/data/docs/specifications/

---
## Mass Filter

So I am looking for galaxies that have enough mass to visualize well.  Thus I will be choosing galaxies that are a scale multiple above and below the mass of the Milky Way.

In [6]:
def getMassFilter( args, snapNum, mScale = 10 ):
    
    # Define where file will be saved
    mLoc = f'{args.dataDir}/{args.simName}-{snapNum}-mask-mass-{mScale}.npy'
    
    # Read from file if it exits
    if os.path.exists( mLoc ) and not args.overwrite:
        print(f"Reading Mass Mask: {mLoc}")
        mass_mask = np.load( mLoc )
        return mass_mask
    
    # define mass limits
    milky_way_mass = 150.0  # in (10^10 M_⊙) 
    upper_mass = milky_way_mass * mScale
    lower_mass = milky_way_mass / mScale
    
    # Pull masses for all subhalos in snapshot
    fields = ['SubhaloMass']
    print("Pulling Masses for all Subhalos")
    print("WARNING: May take a while ")
    SubhaloMass = il.groupcat.loadSubhalos( args.simDir, snapNum, fields=fields)
    
    # This is the first occasion where I wi
    
    # Find galaxies between upper and lower mass
    mask_mass = ( SubhaloMass[:] <= upper_mass ) & ( SubhaloMass[:] >= lower_mass )
    
    # Save mass
    np.save( mLoc, mask_mass )
    
    return mask_mass
    
if buildEnv and True:
    args.overwrite = False
    mask_mass = getMassFilter( args, 50 )
    print( mask_mass.shape, mask_mass.dtype )
    mask_mass = getMassFilter( args, 67 )
    print( mask_mass.shape, mask_mass.dtype )


Reading Mass Mask: tng-data/TNG50-1-50-mask-mass-10.npy
(6780233,) bool
Reading Mass Mask: tng-data/TNG50-1-67-mask-mass-10.npy
(6244619,) bool


___
## Centrals and Satellites
Halo's often have a central largest galaxy, with smaller subhalos in orbit around the halo called satellites.  For convenience, let's create a mask of these central galaxies.


In [7]:
def expand_mask_from_list( true_list, snapnum ):    
    n_subhalo = il.groupcat.loadHeader( args.simDir, snapnum)['Nsubgroups_Total']
    mask = np.full( n_subhalo, False, dtype=bool )
    mask[true_list] = True    
    return mask
    

def getCentralFilter( args, snapnum = 99 ):
    
    mLoc = f'{args.dataDir}/{args.simName}-{snapnum}-mask-central.npy'

    # If already obtained, read from file
    if os.path.exists( mLoc ) and not args.overwrite:
        print(f"Reading Central Galaxy file: {mLoc}")
        mask_central = np.load( mLoc )
        return mask_central

    print(f"Getting Central SubHalo IDs for sim/snapshot: {args.simName} / {snapnum}")

    # The GroupFirstSub is the subhalo id for the largest subhalo in a halo.  
    GroupFirstSub = il.groupcat.loadHalos( args.simDir, snapnum, fields=['GroupFirstSub'])

    # Filter out groups that contain no subhalos.
    w = np.where(GroupFirstSub >= 0) # value of -1 indicates no subhalo in this group
    central_ids = GroupFirstSub[w]
    
    # Expand into a full array with a value for every subhalo
    mask_central = expand_mask_from_list( central_ids, snapnum = snapnum )
    
    # Save mass
    np.save( mLoc, mask_central )
    
    return mask_central

if buildEnv and True: 

    args.overwrite = False
    mask_central = getCentralFilter( args, snapnum = 50 )
    print('Central Galaxies:', mask_central.shape, mask_central[:10] )
    mask_central = getCentralFilter( args, snapnum = 67 )
    print('Central Galaxies:', mask_central.shape, mask_central[:10] )
    

Reading Central Galaxy file: tng-data/TNG50-1-50-mask-central.npy
Central Galaxies: (6780233,) [ True False False False False False False False False False]
Reading Central Galaxy file: tng-data/TNG50-1-67-mask-central.npy
Central Galaxies: (6244619,) [ True False False False False False False False False False]


In [8]:
if buildEnv:
    tmp2 = il.groupcat.loadHeader(args.simDir, 67)
    print( tmp2 )
    print( tmp2['Nsubgroups_Total'] )
    print( type(tmp2['Nsubgroups_Total'] ))

{'BoxSize': 35000.0, 'FlagDoubleprecision': 0, 'Git_commit': b'd203ec8b07c7e2bdda5f608aa0babea46d603699', 'Git_date': b'Thu Apr 7 14:14:27 2016 +0200', 'HubbleParam': 0.6774, 'Ngroups_ThisFile': 1, 'Ngroups_Total': 10638943, 'Nids_ThisFile': 11555101, 'Nids_Total': 8283281991, 'Nsubgroups_ThisFile': 54051, 'Nsubgroups_Total': 6244619, 'NumFiles': 680, 'Omega0': 0.3089, 'OmegaLambda': 0.6911, 'Redshift': 0.5030475232448832, 'Time': 0.6653149581332802}
6244619
<class 'numpy.int32'>


# (r) Galaxy Morphologies (Deep Learning)

Because our method relies on disks of galaxies, it might be useful for us to find mergers betweeen two disk galaxies. 

In [9]:
def getDiskMorphologyFilter( args, snapNum = 99 ):
    
    mLoc = f'{args.dataDir}/{args.simName}-{snapNum}-mask-disk-morphology.npy'

    # If already obtained, read from file
    if os.path.exists( mLoc ) and not args.overwrite:
        print(f"Reading Disk Morphology Mask: {mLoc}")
        mask_disk = np.load( mLoc )
        return mask_disk
    
    # Check if morphology file exists.  
    hdf5Loc = f'subcatalogs/TNG50-1-morphologies_deeplearn.hdf5'
    if not os.path.exists( hdf5Loc ):
        print("WARNING:!  Subcatalog File missing: ", hdf5Loc )
        print("Please see IllustrisTNG Data Specs for info to download this file.")
        raise AssertionError
    
    # Read the deeplearning morphology file
    with h5py.File(f'subcatalogs/TNG50-1-morphologies_deeplearn.hdf5', 'r') as file:
        
        header = f'Snapshot_{snapNum}'
        
        # Verify snapshot header is in file
        if header not in file.keys():
            print(f"Bad HDF5 header: {header} / {file.keys()}" )
            return None       
        
        subhaloIDs      = np.array( file[header]['SubhaloID'] )
        subhaloDiskProb = np.array( file[header]['P_Disk'] )
        
        print( 'test morph: ', subhaloIDs.shape )
        
        # Iterate through and grab subhalos with a greater chance of being a disk galaxy
        disk_list = []
        for i in range( subhaloIDs.shape[0] ):
            if subhaloDiskProb[i] > 0.5:
                disk_list.append( subhaloIDs[i] )
        
    # Done reading file.
    
    # create mask 
    mask_disk = expand_mask_from_list( np.array( disk_list ), snapNum )
    
    # Save mass
    print(f"Saving Disk Morphology Mask: {mLoc}")
    np.save( mLoc, mask_disk )
    
    return mask_disk    


if buildEnv and True:
    tf.explore_hdf5( 'subcatalogs/TNG50-1-morphologies_deeplearn.hdf5', max_depth = 0 )
    # args.overwrite=False
    # mask_disk = getDiskMorphologyFilter( args, 67 )  
    # print('Disk Galaxies:', mask_disk.shape, mask_disk[:10] )
    # mask_disk = getDiskMorphologyFilter( args, 50 )  
    # print('Disk Galaxies:', mask_disk.shape, mask_disk[:10] )

{'Header': {'contents': {}, 'type': 'Group'},
 'Snapshot_25': {'contents': {}, 'type': 'Group'},
 'Snapshot_29': {'contents': {}, 'type': 'Group'},
 'Snapshot_33': {'contents': {}, 'type': 'Group'},
 'Snapshot_40': {'contents': {}, 'type': 'Group'},
 'Snapshot_50': {'contents': {}, 'type': 'Group'},
 'Snapshot_67': {'contents': {}, 'type': 'Group'}}


## (y) Merger History

Because manually detecting major mergers in the merger tree is messy (trust me, I tried), I'll be using someone else's subcatalogs to detect major mergers between galaxies.   

In [10]:

def getMajorMergerMask( args, snapNum = 67, snapCutoff=13 ):
    
    mLoc = f'{args.dataDir}/{args.simName}-{snapNum}-mask-major-merger-{snapCutoff}.npy'
    
    # If already obtained, read from file
    if os.path.exists( mLoc ) and not args.overwrite:
        print(f"Reading Upcoming Major Merger Mask: {mLoc}")
        mask_merger = np.load( mLoc )
        return mask_merger
    
    file_loc = f'subcatalogs/MergerHistory_0{snapNum}.hdf5'
    
    print( f"Merger History Loc: {file_loc}" )
    
    # Return None if no file found for snap num.
    if not os.path.exists( file_loc ):
        print(f"WARNING:  Could not find file: {file_loc}")
        raise ValueError(f"Subcatalog File Missing: {file_loc}")
        return None
    
    # Read Merger History file.    
    with h5py.File(file_loc, 'r') as file:
            
        # Get the object (could be a group or dataset)
        dataset = file['SnapNumNextMajorMerger']
        
        # Create a boolean mask for values that are non-negative and below the upper limit
        mask_merger = (dataset[:] >= 0) & (dataset[:] <= (snapNum + snapCutoff) )
        
    # Saving
    print(f"Saving Major Merger Mask: {mLoc}")
    np.save( mLoc, mask_merger )
       
    return mask_merger
            
    # Find merger
    
# Get merger tree catalog


if buildEnv and True:
    
    # print_HDF5_info( f'subcatalog/MergerHistory_0{snapNum}.hdf5' )
    
    mask_merger = getMajorMergerMask( args, 50, 18 )
    print('Disk Galaxies:', mask_merger.shape, mask_merger[:10] )
    mask_merger = getMajorMergerMask( args, 67, 33 )
    
    print('Disk Galaxies:', mask_merger.shape, mask_merger[:10] )
    
        

Reading Upcoming Major Merger Mask: tng-data/TNG50-1-50-mask-major-merger-18.npy
Disk Galaxies: (6780233,) [ True False False False False False False False  True False]
Reading Upcoming Major Merger Mask: tng-data/TNG50-1-67-mask-major-merger-33.npy
Disk Galaxies: (6244619,) [ True False False False False False False False False False]


---
## Combine Masks and Generate MOI_1 IDs

(Matt from the future here) 
This is step one of many to narrow down potential mergers so I'm going to establish some terminology as I'm rewriting this.  potential mergers will hence forth be furthered as Mergers-of-Interest or MOI for short.  And each step will have a number, so this will be moi_1.  

### MOI_1: Parent galaxy that will collide soon.

In [11]:
def combine_masks(mask_list):
    
    # Verify that all masks have the same shape
    if not all(mask.shape == mask_list[0].shape for mask in mask_list):
        print("ERROR:  Masks do not have the same shape.")
        for mask in mask_list:
            print( len(mask) )
        raise ValueError("ERROR: Combine Masks: All masks must have the same shape")

    # Initialize the combined mask with the first mask
    combined_mask = mask_list[0].copy()

    # Perform logical AND operation with each subsequent mask
    for mask in mask_list[1:]:
        combined_mask &= mask

    return combined_mask

def generate_moi_1_ids( args, snapNum, mass = True, massScale = 10, central = False, disk = True, major = True, majorCutoff = 13 ):
    
    # Create list of mask to find goi
    mask_list = []
    if mass:     mask_list.append( getMassFilter           ( args, snapNum, mScale = massScale ) )
    if central:  mask_list.append( getCentralFilter        ( args, snapNum ) )
    if disk:     mask_list.append( getDiskMorphologyFilter ( args, snapNum ) )
    if major:    mask_list.append( getMajorMergerMask      ( args, snapNum, majorCutoff ) )

    try:
        # Get 
        combined_mask = combine_masks( mask_list )
        subhalo_ids = np.where( combined_mask )[0] 
        
    except ValueError as e:
        print(e)
        return None
    
    # Loop through subhalo ids and combine with snapnum to create SubhaloIDRaw
    moi_1_ids = []
    
    for mid in subhalo_ids:
        moi_1_ids.append( tf.generate_subhalo_id_raw( snapNum, mid, )  )
    
    moi_1_ids = np.array( moi_1_ids )
    
    return moi_1_ids


    
    
if buildEnv and True:
    

    # Starting snaps and their ranges for looking
    #  (based on merger history hdf5 file)
    snap_info = { 
        25: {'start_snap':25, 'stop_snap':29}, 
        29: {'start_snap':29, 'stop_snap':33}, 
        33: {'start_snap':33, 'stop_snap':40},
        40: {'start_snap':40, 'stop_snap':55}, 
        50: {'start_snap':50, 'stop_snap':67}, 
        67: {'start_snap':67, 'stop_snap':99}
    }
    
    moi_1_ids = []
    args.overwrite = False
        
    for snap in snap_info:

        # Initial GOIs of interest
        snap_info[snap]['moi_1'] = generate_moi_1_ids( args, snap, majorCutoff = snap_info[snap]['stop_snap']- snap_info[snap]['start_snap']) 
    
    c = 0
    for s in snap_info:
        c += len( snap_info[s]['moi_1'] )
    
    print("done")
    print( "MOI_1 Found: ", c )


Reading Mass Mask: tng-data/TNG50-1-25-mask-mass-10.npy
Reading Disk Morphology Mask: tng-data/TNG50-1-25-mask-disk-morphology.npy
Reading Upcoming Major Merger Mask: tng-data/TNG50-1-25-mask-major-merger-4.npy
Reading Mass Mask: tng-data/TNG50-1-29-mask-mass-10.npy
Reading Disk Morphology Mask: tng-data/TNG50-1-29-mask-disk-morphology.npy
Reading Upcoming Major Merger Mask: tng-data/TNG50-1-29-mask-major-merger-4.npy
Reading Mass Mask: tng-data/TNG50-1-33-mask-mass-10.npy
Reading Disk Morphology Mask: tng-data/TNG50-1-33-mask-disk-morphology.npy
Reading Upcoming Major Merger Mask: tng-data/TNG50-1-33-mask-major-merger-7.npy
Reading Mass Mask: tng-data/TNG50-1-40-mask-mass-10.npy
Reading Disk Morphology Mask: tng-data/TNG50-1-40-mask-disk-morphology.npy
Reading Upcoming Major Merger Mask: tng-data/TNG50-1-40-mask-major-merger-15.npy
Reading Mass Mask: tng-data/TNG50-1-50-mask-mass-10.npy
Reading Disk Morphology Mask: tng-data/TNG50-1-50-mask-disk-morphology.npy
Reading Upcoming Major M

--- 
## Find MOI_2 in Future Merger Trees

Since requesting a merger tree only returns it's tree for the current moment and backwards in time, I need to jump several snapshots forward, and identify which galaxies it belongs to there.  It's a long tedious process but I'll figure it out.

### MOI_2:  Child galaxy that has already undergone a major merger event

In [13]:
def find_mois_in_tree( tree_id_raw, moi_1_list, args, ):
    
    tree_snap, tree_id = tf.deconstruct_subhalo_id_raw( tree_id_raw )
    
    # Load only SubhaloIDRaw for effecient retrieval time
    tree = il.sublink.loadTree( args.simDir, tree_snap, tree_id, fields=['SubhaloIDRaw'] )

    # See if any of my MOIs are in list
    moi_1_mask = np.isin( moi_1_list, tree )
    n_matches = np.sum(moi_1_mask) # Presuming True = 1 and False = 0

    # If none found
    if n_matches == 0:
        return []
    # Else we have results

    # Get the index locations where the mask is True
    moi_1_loc = np.where(moi_1_mask)[0]
    moi_1_ids_in_tree = moi_1_list[ moi_1_loc ]

    moi_2_list = []

    for moi_1 in moi_1_ids_in_tree:
        moi_2_list.append( ( tree_id_raw, moi_1 ) )

    return moi_2_list


def getMOI_2( args, snap_info, func_overwrite = False ):
    
    mLoc = f'{args.dataDir}/{args.simName}-moi_2.txt'
    
    # If already obtained, read from file
    if os.path.exists( mLoc ) and not args.overwrite and not func_overwrite:
        print(f"Reading Merger-of-Interest List: {mLoc}")
        moi_list = np.loadtxt( mLoc, dtype=int )
        return moi_list
    
    # Loop through snaps of interest, collect moi_2 value
    
    for snap in snap_info:
        
        print(f"Snap {snap} of {snap_info.keys()}")
        stop = snap_info[snap]['stop_snap']
        moi_1_list = snap_info[snap]['moi_1']
    
        # Basically just get lots of galaxies above mass range in final snapshot
        tree_id_list = generate_moi_1_ids( args, stop, \
                                               mass = True, massScale = 50, \
                                               central = False, disk = False, \
                                               major=False )
        # Find moi_2 containing moi_1
        moi_2_list = []
        for i, tree_id_raw in enumerate(tree_id_list):  
            tree_snap, tree_id = tf.deconstruct_subhalo_id_raw( tree_id_raw )
            tabprint( f" {i} / {tree_id_list.shape[0]} - {tree_id_raw} - Found: {len(moi_2_list)}", end='\r' )

            found_ids = find_mois_in_tree( tree_id_raw, moi_1_list, args )
            moi_2_list.extend( found_ids )

        print( f"\nFound MOI_2 / Tree Matches: {len( moi_2_list) }")
        
        snap_info[snap]['moi_2'] = moi_2_list
        
    np.savetxt( mLoc, moi_2_list )
    
    return snap_info

if buildEnv and True:
    
    args.overwrite=False
    snap_info = getMOI_2( args, snap_info, True )
    
    print( type( snap_info ) )
    print( snap_info[67]['moi_2'] )
        


Snap 25 of dict_keys([25, 29, 33, 40, 50, 67])
Reading Mass Mask: tng-data/TNG50-1-29-mask-mass-50.npy
	 -  2990 / 2991 - 29000000414949 - Found: 22
Found MOI_2 / Tree Matches: 22
Snap 29 of dict_keys([25, 29, 33, 40, 50, 67])
Reading Mass Mask: tng-data/TNG50-1-33-mask-mass-50.npy
	 -  3379 / 3380 - 33000000482584 - Found: 46
Found MOI_2 / Tree Matches: 46
Snap 33 of dict_keys([25, 29, 33, 40, 50, 67])
Pulling Masses for all Subhalos
	 -  3805 / 3806 - 40000000554961 - Found: 95
Found MOI_2 / Tree Matches: 95
Snap 40 of dict_keys([25, 29, 33, 40, 50, 67])
Pulling Masses for all Subhalos
	 -  4096 / 4097 - 55000000642360 - Found: 188
Found MOI_2 / Tree Matches: 188
Snap 50 of dict_keys([25, 29, 33, 40, 50, 67])
Reading Mass Mask: tng-data/TNG50-1-67-mask-mass-50.npy
	 -  4137 / 4138 - 67000000686265 - Found: 196
Found MOI_2 / Tree Matches: 196
Snap 67 of dict_keys([25, 29, 33, 40, 50, 67])
Reading Mass Mask: tng-data/TNG50-1-99-mask-mass-50.npy
	 -  4095 / 4096 - 99000000791758 - Found

## MOI_3:  MOI_2 with multiple MOI_1 in their tree

In [16]:
def find_moi_2_repeats( moi_list ):
    
    # Step 1: Extract all moi_2's from the tuples
    moi_2_list = [mois[0] for mois in moi_list]

    # Step 2: Count the occurrences of each moi_2
    moi_2_counts = Counter(moi_2_list)

    # Step 3: Filter the tuples where moi_2 appears more than once
    moi_2_repeats = [mois for mois in moi_list if moi_2_counts[mois[0]] > 1]
    
    return moi_2_repeats

if buildEnv and True:
    
    m2_duos = []
    
    for snap in snap_info:
        
        print(snap )
    
        m2_duos.extend( find_moi_2_repeats( snap_info[snap]['moi_2'] ) )

    print( 'MOI 2 Duos: ', len( m2_duos ) )

25
29
33
40
50
67
MOI 2 Duos:  126


In [17]:

# Define print fucntion for a row
def printRow( tree, i, fields ):
    # if i == -1:
    #     print("Invalid index")
    #     return
    
    print( " - ".join( [ f"{key}:{tree[key][i]}" for key in fields ]) )
    
def createVisLink( subhaloIDRaw, projection = 'face', simulation='TNG50-1' ):
        tmp = tf.deconstruct_subhalo_id_raw( subhaloIDRaw )    
        
        if projection == 'face':
            link = f"https://www.tng-project.org/api/{simulation}/snapshots/{tmp[0]}/subhalos/{tmp[1]}/vis.png?partType=stars&partField=stellarComp-jwst_f200w-jwst_f115w-jwst_f070w&size=1&method=histo&rotation=face-on&plotStyle=edged"
        # Else, project x,y plane
        else:
            link = f"https://www.tng-project.org/api/{simulation}/snapshots/{tmp[0]}/subhalos/{tmp[1]}/vis.png?partType=stars&partField=stellarComp-jwst_f200w-jwst_f115w-jwst_f070w&size=1&method=histo&nPixels=256%2C256&axes=0%2C1&plotStyle=edged"
        return link

def get_merger_snapshot( id_raw ):
    
    snap, subhalo_id = tf.deconstruct_subhalo_id_raw( id_raw )
    
    merger_loc = f'subcatalogs/MergerHistory_0{snap}.hdf5'
    
    # Return None if no file found for snap num.
    if not os.path.exists( merger_loc ):
        print(f"WARNING:  Could not find file: {merger_loc}")
        raise ValueError(f"Subcatalog File Missing: {merger_loc}")
        return None
    
    # Read Merger History file.    
    with h5py.File(merger_loc, 'r') as file:
            
        # Get the object (could be a group or dataset)
        dataset = file['SnapNumNextMajorMerger']
        
        merger_snap = dataset[subhalo_id]
        
    
    return merger_snap

if buildEnv:
    tmp = get_merger_snapshot( m2_duos[0][1] )
    print(' Snap: ', tmp )

 Snap:  28


In [18]:

def get_moi_info( args, moi_info ):
    
    m2_id = moi_info[0]
    m1_id = moi_info[1]
        
    # Fields to load from Tree
    fields = ['SubhaloID','NextProgenitorID','MainLeafProgenitorID','FirstProgenitorID','SubhaloGrNr',\
              'SubhaloIDRaw','SubhaloMass', 'RootDescendantID', 'SnapNum', 'DescendantID',\
              'SubhaloPos', 'SubhaloVel', 'SubhaloSpin', 'SubhaloHalfmassRad', ]
    
    m2_snap, m2_subhalo = tf.deconstruct_subhalo_id_raw( m2_id )
    m1_snap, m1_subhalo = tf.deconstruct_subhalo_id_raw( m1_id )   
    
    # Get snapshot when moi is supposed to occur
    merger_snap = get_merger_snapshot( m1_id )
    
    # Load Tree with desired fields
    tree = il.sublink.loadTree( args.simDir, m2_snap, m2_subhalo, fields=fields)
    
    # Create a dictionary to map Subhalo IDs to their index in the list
    tree_index = {subhalo_id: index for index, subhalo_id in enumerate(tree['SubhaloID'])}  
    ci = 0  # Starting index of requested subhalo/snapshot
    
    # Grab ids and velocity arrays of the primary galaxies throughout time. 
    pVel = np.ones( (m2_snap+1, 3) ) * np.nan    # Velocities
    pIDRaw = np.zeros( ( m2_snap+1), dtype=int)  # SubhaloIDRaw
        
    while ci != -1:        
        i, tmp = tf.deconstruct_subhalo_id_raw( tree['SubhaloIDRaw'][ci] )
        
        # Grab array values
        pVel[i,:] = tree['SubhaloVel'][ci][:]
        pIDRaw[i] = tree['SubhaloIDRaw'][ci]
        
        # Update to primary parent in previous snapshot
        ci = tree_index.get( tree['FirstProgenitorID'][ci], -1 )

    # Calculate the change in velocity (Δv)
    dVel = np.diff(pVel, axis=0)    
    # Calculate magnitude of acceleration at each time step (assumption Δt=1)
    pAcc = np.sqrt( np.sum( dVel**2, axis=-1 ) )
    snapshots = [ tf.deconstruct_subhalo_id_raw( pid )[0] for pid in pIDRaw ]
    
    snaploc = 1000000000000
    
    data_list = []
    
    # Iterate through snapshots to get info
    for snap in range( m1_snap-5, m2_snap ):
        
        # Find rows for current snapnum
        snap_mask = (tree['SubhaloIDRaw'] // snaploc) % snaploc == snap
        snap_index = np.where( snap_mask )

        # Gather masses for snap
        snap_masses = tree['SubhaloMass'][snap_index]

        # Find n highest masses
        n = 2
        top_index = np.argsort(snap_masses)[-n:][::-1]
        
        if len(top_index) <= 1: continue
       
        # Treat 1st and 2nd most massive galaxies as *potential* primary and secondary galaxies
        pid = snap_index[0][top_index[0]]
        sid = snap_index[0][top_index[1]]
        
        keys =  [ 'SubhaloIDRaw', 'SubhaloMass', 'SubhaloPos', 'SubhaloVel', 'SubhaloSpin', 'SubhaloHalfmassRad', ]
        
        # Store values in a flat dictionary
        flat_dict = {}
        flat_dict['snap'] = snap
        flat_dict['moi_1_id'] = m1_id
        flat_dict['moi_2_id'] = m2_id
        flat_dict['merger_snap'] = merger_snap
        flat_dict['p_acceleration'] = pAcc[snap]
        flat_dict['xy_projection'] = createVisLink( tree['SubhaloIDRaw'][pid], projection = 'xy' )
        flat_dict['p_face_projection'] = createVisLink( tree['SubhaloIDRaw'][pid], projection = 'face' )
        flat_dict['s_face_projection'] = createVisLink( tree['SubhaloIDRaw'][sid], projection = 'face' )
        
        # Loop through catalog data and store
        for k in keys:
            for c, ii in [ ('p',pid), ('s',sid) ]:
                # print( k, c, ii )
                flat_dict[f'{c}_{k}'] = tree[k][ii]
        
        data_list.append(flat_dict)
    
    return data_list
        

def save_moi_info( args, moi_list, moi_file, func_overwrite=False ):
        
    # If file exists, read and return.
    if os.path.exists( moi_file ) and args.overwrite == False and func_overwrite == False:
        df = pd.read_csv( moi_file )
        return df
    
    # Else, create file by getting info via merger trees.
    data_list = []
    n = len( moi_list )
    for i in range( n ):
        print( i, ' / ', n, end='\r'  )
        moi_info = get_moi_info( args, moi_list[i] )
        data_list.extend( moi_info )
        
    print('')
    # Convert list into dataframe
    df = pd.DataFrame(data_list)

    df.to_csv( moi_file , index=False )
    
    # return df
    

if buildEnv and True:
    
    args.overwrite = False
      
    m2_duos_df = save_moi_info( args, m2_duos, f'{args.dataDir}/moi_2-duos.csv', func_overwrite=True )
    
    print( m2_duos_df )

125  /  126
None


# MOI_3:  Manual Search

While it would be nice if there was an automatic way to review these potential targets, I need to find a handful of visually nice targets now.  So I loaded the CSV as an Excel spreadsheet, and manually opened the links to visuallize the galaxies.  For any that show  hints of tidal features, I'm pasting the snapnum and subhalo id down below.

### MOI_3:  Galaxies that show hints of tidal features.

NOTE to myself.  The following is only from reviewing moi_2_50_67_repeats.   More are sure to be found in newer file m2_duos.

In [ ]:

moi_4_great = {
    54:45004,
    61:186579,
    62:190717,   # AMAZING
    63:192216,
    64:153811, 
    62:187573,
}

moi_3_good = [
    (54, 45004),
    (55, 46172),
    (56, 46470),
    (62, 187573),
    (63, 189019),
    (64, 205294),
    (61, 186579),
    (62, 190717),
    (63, 192216),
    (64, 153811),
    (56, 230959),
]


moi_3_maybe = [
    (58, 136813),
    (59, 138897),
    (56, 192189),
    (59, 210457),
    (62, 338866),
    (63, 336934),
    (64, 338830),
    (65, 345132),
    (55, 330164),
    (56, 333739),
    (57, 336505),
]

if buildEnv:
    
    # Let's save some of these test moi-3s somewhere.
    m3_file = 'tng-data/moi_3-test.csv'
    
    # Identify MOI_2 id's that contain our MOI_3 id.
    # Get rows containing moi_3 ids
    pid_list = [ tf.generate_subhalo_id_raw( snap, subhalo_id ) for snap, subhalo_id in moi_3_good ]
    print( pid_list )
    
    # Extract moi_2 ids associated with p_ids from moi_3 list.
    # print( m2_duos_df['p_SubhaloIDRaw'] )
    moi2_ids = m2_duos_df[ m2_duos_df['p_SubhaloIDRaw'].isin(pid_list) ]['moi_2_id'].unique()
    print( moi2_ids )
    
    # Get all rows with moi_2 values, thus getting all info before and after moi_3 snap
    moi_3_df = m2_duos_df[ m2_duos_df['moi_2_id'].isin(moi2_ids) ]
    
    # Add new columns indicating row of MOI_3 candidates
    moi_3_df['moi_3'] = False
    moi_3_df.loc[moi_3_df['p_SubhaloIDRaw'].isin(pid_list), 'moi_3'] = True
    
    moi_3_df.to_csv( m3_file , index=False )
    


In [ ]:
#